In [2]:
#saves the tour index for NA into file viator_north_america.html - is a bit tricky because viator hates you to do that - so you need to solve the captcha manually. Thanks Claude for solving this ;-)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from pathlib import Path

def save_webpage_content(url, output_file="webpage_content.html"):
    try:
        # Set up Chrome options
        chrome_options = Options()
        # Don't run headless - we need to solve CAPTCHA
        chrome_options.add_argument('--start-maximized')
        chrome_options.add_argument('--disable-blink-features=AutomationControlled')
        # Remove automation flags
        chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
        chrome_options.add_experimental_option('useAutomationExtension', False)
        
        # Initialize the driver
        driver = webdriver.Chrome(options=chrome_options)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        try:
            # Get the page
            print("Loading page...")
            driver.get(url)
            
            # Wait for human to solve CAPTCHA
            print("Please solve the CAPTCHA in the browser window...")
            input("Press Enter after solving the CAPTCHA...")
            
            # Wait for main content to load
            time.sleep(5)
            
            # Get the page source after CAPTCHA
            page_source = driver.page_source
            
            # Save the content to a file
            Path(output_file).write_text(page_source, encoding='utf-8')
            print(f"Content successfully saved to {output_file}")
            
        finally:
            # Always close the driver
            driver.quit()
            
    except Exception as e:
        print(f"Error: {e}")

# URL to scrape
url = "https://www.viator.com/sitemap/North-America/d8"

# Call the function
save_webpage_content(url, "viator_north_america.html")

Loading page...
Please solve the CAPTCHA in the browser window...
Content successfully saved to viator_north_america.html


In [4]:
# uses the viator_north_america.html file to itertate through all tours and downlaod the tour details into new files in dir scrpaed_pages/NAtours
# make sure you do not use a server that can be linked to a voicemap IP - generally speaking, you are allowed to scrape this content because it is not behind a paywall or requires user identification, but still, better not to raise any red flags
# run the code twice, sometimes downlaods fail - on the second run, it will NOT try to downlaod already existing files

import requests
from bs4 import BeautifulSoup
import time
import os
import random

# Load the local HTML file
# html_file_path = r'C:\Users\mjsteenberg\Documents\VoiceMap\Viator\viator_north_america.html'  # https://www.viator.com/sitemap/North-America/d8
html_file_path = '/Users/mjsteenberg/Desktop/Desktop - MJ\'s MacBook Air - 1/VM/viator-scraper/viator_north_america.html'
with open(html_file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')

# Find the <ul> with the specific data-automation attribute
ul_tag = soup.find('ul', {'data-automation': 'sitemap-entry-list'})

# Extract the <a> tags within the <li> tags to get the URLs
base_url = 'https://www.viator.com'
urls = []
for li in ul_tag.find_all('li'):
    a_tag = li.find('a')
    if a_tag and 'href' in a_tag.attrs:
        # Append ?sortType=rating to each URL
        full_url = base_url + a_tag['href'] + '?sortType=rating'
        urls.append(full_url)

# Create a directory to save the HTML files
save_dir = '/Users/mjsteenberg/Desktop/Desktop - MJ\'s MacBook Air - 1/VM/viator-scraper/scraped_pages/NAtours'
os.makedirs(save_dir, exist_ok=True)

# Function to extract the city and short part of the URL (useful for filenames)
def extract_filename(url):
    # Split the URL into base and query part
    base_url_part = url.split('?')[0]
    query_part = url.split('?')[-1]

    # Extract the city (second last part) and the short URL (last part)
    url_parts = base_url_part.split('/')
    city_part = url_parts[-2] if len(url_parts) > 1 else ''
    short_url_part = url_parts[-1] or url_parts[-2]

    # Create a meaningful filename combining both city, short URL, and query part
    filename = f"{city_part}_{short_url_part}_{query_part.replace('=', '_')}.html"
    return filename

# Loop over the URLs, scrape the content, and save it to disk
for i, url in enumerate(urls):
    try:
        # Generate the filename
        filename = extract_filename(url)
        file_path = os.path.join(save_dir, filename)
        
        # Skip scraping if the file already exists
        if os.path.exists(file_path):
            print(f"File already exists for {url}, skipping.")
            continue
        
        print(f"Scraping URL {i+1}/{len(urls)}: {url}")
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': 'https://www.google.com/'
        }

        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            # Save the content to an HTML file
            with open(file_path, 'w', encoding='utf-8') as f:
                f.write(response.text)
            print(f"Saved {url} to {file_path}")
        else:
            print(f"Failed to retrieve {url}, Status Code: {response.status_code}")
        
        # Wait for a random time between 1 and 5 seconds before making the next request
        sleep_time = random.uniform(1, 5)
        print(f"Waiting for {sleep_time:.2f} seconds before the next request...")
        time.sleep(sleep_time)
    
    except Exception as e:
        print(f"Error scraping {url}: {e}")


FileNotFoundError: [Errno 2] No such file or directory: "/Users/mjsteenberg/Desktop/Desktop - MJ's MacBook Air - 1/VM/viator-scraper/viator_north_america.html"